## Package imports

In [1]:
import pandas as pd

----
## Display settings

In [2]:
pd.set_option("display.width", 140)

----
## Reading in the data

In [3]:
dtypes = {
    "MORTGAGE": "category",
    "MORTGAGE_GETTING_AGE": "Int64",
    "AGE": "int64",
    "YEARS_WITH_BANK": "int64",
    "MARITAL_STATUS": "category",
    "EDUCATION": "category",
    "EMPLOYMENT": "category",
    "GENDER": "category",
    "INCOME": "float64",
    "BALANCE": "float64",
}

df = pd.read_csv(
    "../data/processed/clean/Retail data.csv",
    sep=";",
    dtype=dtypes,
    parse_dates=["ADDRESS_DATE", "JOB_DATE", "WITH_BANK_DATE"],
)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23983 entries, 0 to 23982
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MORTGAGE              23983 non-null  category      
 1   MORTGAGE_GETTING_AGE  306 non-null    Int64         
 2   AGE                   23983 non-null  int64         
 3   YEARS_WITH_BANK       23983 non-null  int64         
 4   MARITAL_STATUS        23940 non-null  category      
 5   EDUCATION             23983 non-null  category      
 6   EMPLOYMENT            23983 non-null  category      
 7   GENDER                23983 non-null  category      
 8   INCOME                23983 non-null  float64       
 9   ADDRESS_DATE          23775 non-null  datetime64[ns]
 10  JOB_DATE              18212 non-null  datetime64[ns]
 11  WITH_BANK_DATE        23983 non-null  datetime64[ns]
 12  BALANCE               23983 non-null  float64       
dtypes: Int64(1), cat

----
## Estimating when the data was created

In [5]:
# Temporary column for possible data creation years
df["_CREATION_YEAR"] = df["WITH_BANK_DATE"].dt.year + df["YEARS_WITH_BANK"]

In [6]:
# Temporary value containing the year of the data creation
_creation_year: int = df["_CREATION_YEAR"].max()

# Temporary date with the month and day values closest to the actual data creation
_creation_date = pd.Timestamp(df[df["_CREATION_YEAR"] == _creation_year]["WITH_BANK_DATE"].max())

# Year and day corrections
DATA_CREATION = _creation_date + pd.DateOffset(years=_creation_year - _creation_date.year, days=1)

print(DATA_CREATION)

2017-07-03 00:00:00


In [7]:
# Remove temporary column
df.drop("_CREATION_YEAR", axis=1, inplace=True)

----
## Setting the data to past values for people with mortgages
This will ensure that the data represents the life situations when people choose to get mortgages.

### Creating new column with the mortgage getting year

In [8]:
df["MORTGAGE_GETTING_YEAR"] = DATA_CREATION.year - df["AGE"] + df["MORTGAGE_GETTING_AGE"]

In [9]:
# Unique values of the new column
df["MORTGAGE_GETTING_YEAR"].unique()

<IntegerArray>
[2015, 2016, 2014, 2017, <NA>]
Length: 5, dtype: Int64

In [10]:
# First 5 rows
df.head()

,MORTGAGE,MORTGAGE_GETTING_AGE,AGE,YEARS_WITH_BANK,MARITAL_STATUS,EDUCATION,EMPLOYMENT,GENDER,INCOME,ADDRESS_DATE,JOB_DATE,WITH_BANK_DATE,BALANCE,MORTGAGE_GETTING_YEAR
0,Y,50,52,13,M,HGH,PVE,M,909.50,1979-04-18,2010-01-01,2004-03-16,7648.35,2015
1,Y,48,49,11,M,HGH,SFE,M,288.46,1999-12-16,2004-01-20,2005-11-07,30189.90,2016
2,Y,53,55,14,M,BCR,STE,M,1280.53,2005-06-28,1990-09-01,2003-06-25,50553.17,2015
3,Y,64,66,10,M,BCR,OTH,F,620.96,2000-12-22,NaT,2006-12-21,15907.28,2015
4,Y,46,47,9,S,MAS,PVE,F,2239.85,2009-08-01,2013-12-24,2007-08-07,27916.19,2016


In [11]:
# Save a reference to individuals with a mortgage
mortgage_rows = df["MORTGAGE"] == "Y"

### Inflation correction
Source: https://www.raiffeisenzertifikate.at/en/inflation-calculator/

In [12]:
# Multiply monetary columns by the average inflation rate of this time period
for column in ["INCOME", "BALANCE"]:
    df.loc[mortgage_rows, column] = (df.loc[mortgage_rows, column] * 1.0125).round(2)

### Removing dates after getting a mortgage

In [13]:
# Set date columns to NaT after mortgage getting year
for column in ["ADDRESS_DATE", "JOB_DATE"]:
    nat_rows = df["MORTGAGE_GETTING_YEAR"] < df[column].dt.year
    df.loc[nat_rows, column] = pd.NaT

### Rolling back the age values to when people got their mortgages

In [14]:
# Set the age of individuals with a mortgage to the age at the time of the mortgage getting
df.loc[mortgage_rows, "AGE"] = df.loc[mortgage_rows, "MORTGAGE_GETTING_AGE"]

### Rolling back the years with the bank too

In [15]:
# Subtract the difference of current year and mortgage getting year from the years with bank
df.loc[mortgage_rows, "YEARS_WITH_BANK"] -= DATA_CREATION.year - df.loc[mortgage_rows, "MORTGAGE_GETTING_YEAR"]

# Set negative values to 0
df.loc[df["YEARS_WITH_BANK"] < 0, "YEARS_WITH_BANK"] = 0

### Removing columns that are no longer needed

In [16]:
# Drop mortgage getting age and mortgage getting year
df.drop(["MORTGAGE_GETTING_AGE", "MORTGAGE_GETTING_YEAR"], axis=1, inplace=True)

In [17]:
# First 5 rows
df.head()

,MORTGAGE,AGE,YEARS_WITH_BANK,MARITAL_STATUS,EDUCATION,EMPLOYMENT,GENDER,INCOME,ADDRESS_DATE,JOB_DATE,WITH_BANK_DATE,BALANCE
0,Y,50,11,M,HGH,PVE,M,920.87,1979-04-18,2010-01-01,2004-03-16,7743.95
1,Y,48,10,M,HGH,SFE,M,292.07,1999-12-16,2004-01-20,2005-11-07,30567.27
2,Y,53,12,M,BCR,STE,M,1296.54,2005-06-28,1990-09-01,2003-06-25,51185.08
3,Y,64,8,M,BCR,OTH,F,628.72,2000-12-22,NaT,2006-12-21,16106.12
4,Y,46,8,S,MAS,PVE,F,2267.85,2009-08-01,2013-12-24,2007-08-07,28265.14


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23983 entries, 0 to 23982
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   MORTGAGE         23983 non-null  category      
 1   AGE              23983 non-null  int64         
 2   YEARS_WITH_BANK  23983 non-null  int64         
 3   MARITAL_STATUS   23940 non-null  category      
 4   EDUCATION        23983 non-null  category      
 5   EMPLOYMENT       23983 non-null  category      
 6   GENDER           23983 non-null  category      
 7   INCOME           23983 non-null  float64       
 8   ADDRESS_DATE     23772 non-null  datetime64[ns]
 9   JOB_DATE         18211 non-null  datetime64[ns]
 10  WITH_BANK_DATE   23983 non-null  datetime64[ns]
 11  BALANCE          23983 non-null  float64       
dtypes: category(5), datetime64[ns](3), float64(2), int64(2)
memory usage: 1.4 MB
